In [1]:
import os
import gc
import pickle
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.notebook import tqdm

pd.options.display.max_rows = None
pd.options.display.max_columns = None

%matplotlib inline

In [2]:
df = pd.read_csv('features.csv')

In [7]:
df['S_2'] = pd.to_datetime(df['S_2'])

df['tmp'] = df.groupby('customer_ID')['S_2'].diff()
df['tmp'] = df['tmp'].apply(lambda x: x.days)

group = df.groupby('customer_ID')['tmp'].agg(['max', 'min', 'mean', 'std']).reset_index()

rename_dict = {k: f"fe_transaction_days_{k}" for k in ['max', 'min', 'mean', 'std']}
group = group.rename(columns=rename_dict)

In [8]:
group.head()

,customer_ID,fe_transaction_days_max,fe_transaction_days_min,fe_transaction_days_mean,fe_transaction_days_std
0,001404030d4c78b76bf02e809387611bdf6b708a42c589...,49.0,8.0,32.416667,10.900028
1,00a0778eb2980d0fa97a0a7a4b32bb534d00437d30693b...,32.0,28.0,30.416667,1.164500
2,0229ecb6a6eb8a724637bd7d8b988d06562f2441f5112e...,52.0,9.0,29.916667,11.696607
3,0264ec10c09b97662a7b19871e4eaf540050afccf65d26...,32.0,28.0,30.500000,1.087115
4,02f887d9280480f14f8cfe1650661f91b034f087f056b8...,59.0,10.0,30.916667,12.580348


In [9]:
df.head()

,customer_ID,S_2,D_39,D_41,D_42,D_43,D_44,D_45,D_46,D_47,D_48,D_49,D_50,D_51,D_52,D_53,D_54,D_55,D_56,D_58,D_59,D_60,D_61,D_62,D_63,D_64,D_65,D_66,D_68,D_69,D_70,D_71,D_72,D_73,D_74,D_75,D_76,D_77,D_78,D_79,D_80,D_81,D_82,D_83,D_84,D_86,D_87,D_88,D_89,D_91,D_92,D_93,D_94,D_96,D_102,D_103,D_104,D_105,D_106,D_107,D_108,D_109,D_110,D_111,D_112,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,D_130,D_131,D_132,D_133,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,S_3,S_5,S_6,S_7,S_8,S_9,S_11,S_12,S_13,S_15,S_16,S_17,S_18,S_19,S_20,S_22,S_23,S_24,S_25,S_26,S_27,P_2,P_3,P_4,B_1,B_2,B_3,B_4,B_5,B_6,B_7,B_8,B_9,B_10,B_11,B_12,B_13,B_14,B_15,B_16,B_17,B_18,B_19,B_20,B_21,B_22,B_23,B_24,B_25,B_26,B_27,B_28,B_29,B_30,B_31,B_32,B_33,B_36,B_37,B_38,B_39,B_40,B_41,B_42,R_1,R_2,R_3,R_4,R_5,R_6,R_7,R_8,R_9,R_10,R_11,R_12,R_13,R_14,R_15,R_16,R_17,R_18,R_19,R_20,R_21,R_22,R_23,R_24,R_25,R_26,R_27,R_28,tmp
0,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-03-05,0.008376,0.007045,NaN,0.355097,0.381721,0.232541,0.749022,0.931925,0.636798,NaN,NaN,0.008505,0.257918,NaN,1.009888,0.678041,NaN,0.673155,0.279011,0.001999,0.845026,0.112573,CO,R,0.003677,NaN,6.0,0.002893,0.255467,0.013176,0.007019,NaN,0.362555,0.338296,NaN,NaN,0.002008,0.009383,0.406832,0.008304,NaN,0.005456,0.008112,0.009082,NaN,NaN,0.007533,0.009429,0.006783,0.004019,0.002943,0.001115,0.004340,0.001371,0.007033,NaN,NaN,0.002144,NaN,0.006973,NaN,NaN,1.003346,0.003928,1.0,0.464769,0.0,-1.0,0.461045,0.459714,0.0,0.727194,0.428788,0.009761,0.365854,0.008233,1.0,0.009708,1.005559,0.007325,0.007439,0.007083,NaN,0.008561,NaN,NaN,NaN,NaN,NaN,0.008365,0.008801,0.001185,NaN,0.006018,0.009393,0.003585,0.401315,0.002239,0.008523,0.327650,0.127948,NaN,0.048080,0.184423,0.005208,0.408499,0.006978,0.008038,0.003621,0.004491,0.001774,0.853806,0.131347,0.868075,0.972705,0.004930,0.007958,0.523405,0.480283,0.001637,0.006680,1.000940,0.009355,0.901268,0.020332,0.034404,0.182016,0.007657,0.004749,0.027181,0.008233,0.154105,0.008163,0.006533,0.007149,0.423158,NaN,0.407278,0.109469,0.004621,0.005013,0.006692,0.184154,0.009886,0.003749,0.006558,0.004967,0.521413,NaN,0.0,1,0.000358,1.004676,0.000329,0.003749,3.0,NaN,0.330606,0.005743,NaN,0.003428,0.005751,0.507771,0.000728,0.008425,0.008175,0.002113,0.004826,NaN,0.007801,0.005049,1.000701,0.000934,0.001050,0.002937,0.005534,0.002551,0.009574,0.006862,0.007049,0.008520,0.005952,0.008763,0.000349,0.005358,NaN,1.000625,0.001186,NaN
1,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-04-10,0.002836,0.000565,NaN,0.312375,0.134045,0.232197,0.416163,0.939839,0.484000,NaN,0.006556,0.006998,0.253597,NaN,1.008957,0.611879,NaN,0.567915,0.275475,0.009932,0.842110,0.077126,CO,R,0.008699,NaN,6.0,0.005307,0.007638,0.012317,0.003940,NaN,0.290779,0.275103,NaN,0.072436,0.000811,0.009827,0.608082,0.008636,0.005899,0.009630,0.006017,0.001705,NaN,NaN,0.001632,0.001945,0.008371,0.002443,0.002520,0.005529,0.006890,0.009558,0.008599,NaN,NaN,0.008818,NaN,0.007328,NaN,NaN,1.002003,0.006697,1.0,0.477699,0.0,-1.0,0.461896,0.465308,0.0,0.732167,0.576645,0.004936,0.365156,0.008392,1.0,0.008002,1.002762,1.004064,0.004804,0.000823,NaN,0.000696,NaN,NaN,NaN,NaN,NaN,0.006429,0.000438,0.004144,NaN,0.002282,0.003261,0.002704,0.394459,0.004846,0.008937,0.322127,0.008656,NaN,0.289070,0.184495,0.004601,0.509170,0.002461,0.004669,0.004006,0.004215,0.004033,0.303755,0.140477,0.085234,0.969171,0.009193,0.005511,0.533376,0.522688,0.003317,0.007121,0.814569,0.011885,0.901686,0.009887,0.046754,0.328685,0.007138,0.008674,0.016825,0.002511,0.057497,0.012143,0.003472,0.004912,0.426391,NaN,0.407492,0.209688,0.009138,0.006126,0.005744,0.340210,0.002182,0.007864,0.005851,0.004641,0.505322,NaN,0.0,1,0.008105,1.005377,0.004619,0.003541,3.0,NaN,0.282520,0.008473,NaN,0.001009,0.005574,0.408810,0.005981,0.003779,0.000730,0.007040,0.003869,NaN,0.009881,0.007829,1.003468,0.009944,0.002582,0.000130,0.005474,0.006805,0.008637,0.0002

In [10]:
df.shape

(5569, 191)

In [58]:
df['S_2'] = pd.to_datetime(df['S_2'])

df['tmp'] = df.groupby('customer_ID')['S_2'].diff()
df['tmp'] = df['tmp'].apply(lambda x: x.days)

group = df.groupby('customer_ID')['tmp'].agg(['max', 'min', 'mean']).reset_index()

In [55]:
df[['customer_ID', 'S_2', 'tmp']].head(20)

,customer_ID,S_2,tmp
0,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-03-05,NaN
1,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-04-10,36.0
2,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-05-11,31.0
3,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-06-26,46.0
4,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-07-27,31.0
5,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-08-04,8.0
6,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-09-09,36.0
7,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-10-07,28.0
8,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-11-25,49.0
9,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-12-26,31.0


In [59]:
group.head()

,customer_ID,max,min,mean
0,001404030d4c78b76bf02e809387611bdf6b708a42c589...,49.0,8.0,32.416667
1,00a0778eb2980d0fa97a0a7a4b32bb534d00437d30693b...,32.0,28.0,30.416667
2,0229ecb6a6eb8a724637bd7d8b988d06562f2441f5112e...,52.0,9.0,29.916667
3,0264ec10c09b97662a7b19871e4eaf540050afccf65d26...,32.0,28.0,30.500000
4,02f887d9280480f14f8cfe1650661f91b034f087f056b8...,59.0,10.0,30.916667


In [50]:
a = df.groupby('customer_ID')[['S_2']].nunique().reset_index()

a.head(50)

,customer_ID,S_2
0,001404030d4c78b76bf02e809387611bdf6b708a42c589...,13
1,00a0778eb2980d0fa97a0a7a4b32bb534d00437d30693b...,13
2,0229ecb6a6eb8a724637bd7d8b988d06562f2441f5112e...,13
3,0264ec10c09b97662a7b19871e4eaf540050afccf65d26...,13
4,02f887d9280480f14f8cfe1650661f91b034f087f056b8...,13
5,03cfbaf2318a08a054f62a5844b15e666f5a57792d0807...,13
6,03d1440738c410f87ed7ca2b4b4b3b48d677fe9ee98e2a...,13
7,049880a002ce667556644e89ebbf01106867d1a5def2ca...,2
8,0499e881bc25d3026288629a25c19cb062cb99e8b3ca3e...,10
9,05f0abfb299925d0412467837dfeff53333a7cbca6a6c7...,13


In [52]:
class GroupbyIDTransformer:
    def __init__(self, feats: list, aggs: list):
        self.feats = feats
        self.aggs = aggs

    def _unique_customer_id(self, df):

        target_df = pd.DataFrame({
            'customer_ID': df['customer_ID'].unique()
        })

        return target_df
    
    def transform(self, df):
        
        target = self._unique_customer_id(df)
        
        for agg in self.aggs:
            group = df.groupby('customer_ID')[self.feats].agg(agg).reset_index()
            rename_dict = {k: f"fe_group_{agg}_key_customer_ID_feat_{k}" for k in self.feats}
            group = group.rename(columns=rename_dict)
            
            target = pd.merge(target, group, on='customer_ID')
            
        return target
    
    



In [53]:
feats = ['D_39', 'D_41']
aggs = ['max', 'mean']
group_id = 'customer_ID'


tt = GroupbyTransformer(feats, aggs, group_id)
res = tt.transform(df)

In [54]:
res.head()

,customer_ID,fe_group_max_key_customer_ID_feat_D_39,fe_group_max_key_customer_ID_feat_D_41,fe_group_mean_key_customer_ID_feat_D_39,fe_group_mean_key_customer_ID_feat_D_41
0,001404030d4c78b76bf02e809387611bdf6b708a42c589...,0.177261,0.008917,0.028039,0.004898
1,00a0778eb2980d0fa97a0a7a4b32bb534d00437d30693b...,0.009428,0.009959,0.005560,0.005816
2,0229ecb6a6eb8a724637bd7d8b988d06562f2441f5112e...,0.242850,0.008297,0.052441,0.004825
3,0264ec10c09b97662a7b19871e4eaf540050afccf65d26...,0.008270,0.009098,0.004711,0.005395
4,02f887d9280480f14f8cfe1650661f91b034f087f056b8...,0.888410,0.798163,0.467966,0.342802


In [42]:
feats = ['D_39', 'D_41']
agg = 'max'
group_id = 'customer_ID'

group = df.groupby(group_id)[feats].agg(agg).reset_index()
rename_dict = {k: f"fe_group_{agg}_key_{group_id}_feat_{k}" for k in feats}
group = group.rename(columns=rename_dict)


In [47]:
group.head(2)

,customer_ID,fe_group_max_key_customer_ID_feat_D_39,fe_group_max_key_customer_ID_feat_D_41
0,001404030d4c78b76bf02e809387611bdf6b708a42c589...,0.177261,0.008917
1,00a0778eb2980d0fa97a0a7a4b32bb534d00437d30693b...,0.009428,0.009959


In [49]:
df[df['customer_ID'] == '001404030d4c78b76bf02e809387611bdf6b708a42c5890ecc057970b51bff87']['D_41'].max()

0.008916918

In [25]:
g_cols = ['customer_ID']
value_cols = ['D_39', 'D_41']
aggs = ['max', 'mean', 'min']
id_col = 'customer_ID'

transformer = GroupbyTransformer(g_cols, value_cols, id_col, aggs)

res = transformer(df)

  0%|          | 0/6 [00:00<?, ?it/s]

ValueError: The column label 'customer_ID' is not unique.

In [24]:
tmp_fe = ['customer_ID', 'S_2', 'D_39', 'D_41'] + [f for f in res.columns if f.startswith('fe')]

res[tmp_fe].head(20)

,customer_ID,S_2,D_39,D_41,fe_group_S_2_D_39_max,fe_group_S_2_D_39_mean,fe_group_S_2_D_39_min,fe_group_S_2_D_41_max,fe_group_S_2_D_41_mean,fe_group_S_2_D_41_min
0,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-03-05,0.008376,0.007045,1.592170,0.417262,0.001904,0.509461,0.133313,0.001378
1,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-04-10,0.002836,0.000565,0.714186,0.139967,0.000923,0.231801,0.045357,0.000565
2,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-05-11,0.005452,0.004533,0.594557,0.101396,0.000807,0.256791,0.033994,0.000195
3,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-06-26,0.008479,0.005404,0.591397,0.141814,0.000405,0.315452,0.022680,0.000736
4,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-07-27,0.001563,0.002158,0.592593,0.120277,0.001142,0.261293,0.020568,0.000109
5,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-08-04,0.005510,0.003301,0.567441,0.117638,0.000672,0.009785,0.005538,0.000024
6,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-09-09,0.177261,0.006952,0.920613,0.138126,0.000659,0.682051,0.072125,0.000542
7,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-10-07,0.123322,0.006850,0.859203,0.237124,0.000650,0.009959,0.006855,0.004502
8,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-11-25,0.009760,0.002129,0.444210,0.050675,0.000330,0.515833,0.028976,0.000005
9,001404030d4c78b76bf02e809387611bdf6b708a42c589...,2017-12-26,0.004978,0.005198,0.565063,0.123801,0.001944,0.149337,0.014799,0.000784
